# Autorisations d'urbanisme en Bretagne (2014-2024)

*Denis Vannier - Le Plan Studio - 2024 - https://leplan.studio/ - https://github.com/Denis-Vannier*

**Source : Ministère des territoires, de l’écologie et du logement, SITADEL** https://www.statistiques.developpement-durable.gouv.fr/liste-des-permis-de-construire-et-autres-autorisations-durbanisme

La version utilisée ici a été géolocalisée par Koumoul :
- Sitadel - Permis de construire et déclarations préalables de logements depuis 2013 : https://opendata.koumoul.com/datasets/sitadel-logements
- Sitadel - Permis de construire et déclarations préalables de locaux depuis 2013 : https://opendata.koumoul.com/datasets/sitadel-locaux 

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.9f' % x)
import requests
import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
import mapclassify
import os
import numpy as np
import shapely
from shapely.geometry import Point, Polygon

In [2]:
dep=["22","29","35","56","44"]
comFR=gpd.read_file("ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2023-12-00156/ADE_3-2_SHP_LAMB93_FXX-ED2023-12-19/COMMUNE.shp")
comFR=comFR[["INSEE_DEP","INSEE_COM","NOM","SIREN_EPCI","geometry"]]
comFR["INSEE_DEP"]=comFR["INSEE_DEP"].astype("str")
Com_BZH=comFR[comFR["INSEE_DEP"].isin(dep)]

epciFR=gpd.read_file("ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2023-12-00156/ADE_3-2_SHP_LAMB93_FXX-ED2023-12-19/EPCI.shp")
epciFR=epciFR[["CODE_SIREN","NOM"]]
epciFR.rename(columns={"CODE_SIREN":"SIREN_EPCI", "NOM":"NOM_EPCI"}, inplace=True)
epciFR["SIREN_EPCI"]=epciFR["SIREN_EPCI"].astype("str")
Com_BZH["SIREN_EPCI"]=Com_BZH["SIREN_EPCI"].astype("str")
Com_BZH=Com_BZH.merge(epciFR, on="SIREN_EPCI", how="left")
Com_BZH.rename(columns={"NOM":"NOM_COMMUNE", "INSEE_DEP":"DEP"}, inplace=True)
Com_BZH=Com_BZH[["DEP","INSEE_COM","NOM_COMMUNE","SIREN_EPCI", "NOM_EPCI"]]
Com_BZH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414 entries, 0 to 1413
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DEP          1414 non-null   object
 1   INSEE_COM    1414 non-null   object
 2   NOM_COMMUNE  1414 non-null   object
 3   SIREN_EPCI   1414 non-null   object
 4   NOM_EPCI     1411 non-null   object
dtypes: object(5)
memory usage: 55.4+ KB


/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Logements Sitadel

In [3]:
Koumoul_loge=pd.read_csv("SITADEL/SITADEL_Koumoul_2014_2024/Koumoul_sitadel-logements.csv")
Koumoul_loge.rename(columns={"COMM":"INSEE_COM", "DEP_CODE":"DEP", "DATE_REELLE_AUTORISATION":"DATE", "NUM_DAU":"NUM_PERMIS", "ETAT_DAU":"ETAT_AVANCEMENT", "CODPOST_DEM":"CP_DEMANDEUR", "LOCALITE_DEM":"COMMUNE_DEMANDEUR", "ADR_NUM_TER":"NUM_VOIE_TERRAIN", "ADR_TYPEVOIE_TER":"TYPE_VOIE-TERRAIN", "ADR_LIBVOIE_TER":"NOM_VOIE_TERRAIN", "ADR_CODPOST_TER":"CP_TERRAIN", "ADR_LOCALITE_TER":"COMMUNE_TERRAIN", "SUPERFICIE_TERRAIN":"SUPERF_TERRAIN", "NATURE_PROJET_DECLAREE":"NATURE_PROJET", "NATURE_PROJET_COMPLETEE":"NATURE_PROJET_DETAIL", "DESTINATION_PRINCIPALE":"DESTINATION", "SURF_HAB_CREEE":"SURF_HABIT_NEW", "SURF_LOC_CREEE":"SURF_LOCAUX_NEW", "NB_LGT_TOT_CREES":"NB_LOGEMENTS_NEW", "RES_PRINCIP_OU_SECOND":"TYPE_HABIT_PRINCIP_SECOND", "APE_DEM":"CODE_ACTIV_DEMANDEUR", "CAT_DEM":"CAT_JURID_DEMANDEUR", "DENOM_DEM":"NOM_DEMANDEUR", "SIREN_DEM":"SIREN_DEMANDEUR", "SIRET_DEM":"SIRET_DEMANDEUR"}, inplace=True)

# On ne retient que les nouvelles constructions : 
Koumoul_loge["NATURE_PROJET"]=Koumoul_loge["NATURE_PROJET"].astype("str")
Koumoul_loge=Koumoul_loge[Koumoul_loge["NATURE_PROJET"]=="1"]

#### Création d'une nouvelle variable permettant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
Koumoul_loge.loc[Koumoul_loge["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
Koumoul_loge.loc[Koumoul_loge["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
Koumoul_loge.drop(columns="ETAT_AVANCEMENT", inplace=True)


#### Création d'une nouvelle variable permettant de savoir 
# si le demandeur a déclaré qu'il ferait du logement construit sa résidence secondaire :
Koumoul_loge.loc[Koumoul_loge["TYPE_HABIT_PRINCIP_SECOND"].astype("str")=="2", "RESID_SECOND"]="1"
Koumoul_loge["TYPE_HABIT_PRINCIP_SECOND"]=Koumoul_loge["TYPE_HABIT_PRINCIP_SECOND"].fillna("0")

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
Koumoul_loge["DATE"]=Koumoul_loge["DATE"].astype("str")
Koumoul_loge["ANNEE_MOIS_AUTORISATION"]=(Koumoul_loge["DATE"].str[:7])+"-01 01:00:00"
Koumoul_loge["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(Koumoul_loge["ANNEE_MOIS_AUTORISATION"])

# Modifier les codes insee des communes (qui ont pu changer depuis 2013 suite aux fusions de communes), 
# en utilisant la table de passage (https://www.insee.fr/fr/information/7671867):
Koumoul_loge["INSEE_COM"]=Koumoul_loge["INSEE_COM"].astype("str")
Koumoul_loge.replace({"INSEE_COM": {"22154":"22093", "22173":"22093", "22038":"22206", "22100":"22219", "22078":"22264", "22247":"22264", "22253":"22264", "22123":"22050", "22007":"22055", "22167":"22107", "22290":"22107", "22298":"22158", "22192":"22209", "22357":"22209", "22066":"22046", "22102":"22046", "22191":"22046", "22292":"22046", "22297":"22046", "22303":"22046", "22051":"22084", "22151":"22093", "22058":"22183", "22080":"22203", "22367":"22251", "29219":"29199", "29129":"29227", "29203":"29021", "29149":"29076", "29052":"29003", "29127":"29266", "35112":"35062", "35113":"35004", "35303":"35004", "35341":"35004", "35100":"35163", "35301":"35184", "35053":"35220", "35269":"35282", "35293":"35282", "35348":"35282", "35011":"35292", "35147":"35308", "35344":"35308", "35209":"35069", "35254":"35069", "35048":"35168", "35083":"35191", "35323":"35191", "35267":"35257", "35158":"35060", "35129":"35176", "44160":"44003", "44017":"44180", "44093":"44180", "44191":"44180", "44219":"44180", "44040":"44005", "44059":"44021", "44008":"44029", "44181":"44087", "44004":"44163", "44011":"44213", "44034":"44213", "44147":"44213", "44060":"49160", "56049":"56213", "56059":"56102", "56138":"56165", "56016":"56173", "56183":"56033", "56038":"56061", "56064":"56061", "56142":"56144", "56192":"56144", "56037":"56197", "56187":"56197", "56150":"56251"}}, inplace=True)

# Ajout du statut et de l'Epci de rattachement des communes :
Com_BZH=Com_BZH[["INSEE_COM","NOM_COMMUNE","SIREN_EPCI","NOM_EPCI"]]
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
Koumoul_loge=Koumoul_loge.merge(Com_BZH, how="left", on="INSEE_COM")


#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)
# On extrait une liste propre des communes :
Detail_Communes=Koumoul_loge[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok", "SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)
# Puis on fusionne avec la base Sitadel :
Koumoul_loge=Koumoul_loge.merge(Detail_Communes, how="left", on="COMMUNE_DEMANDEUR")
Koumoul_loge.replace({"EPCI_DEMANDEUR": {"NR": "0"}}, inplace=True)
Koumoul_loge["EPCI_DEMANDEUR"]=Koumoul_loge["EPCI_DEMANDEUR"].fillna("0").astype("int64")

# Le demandeur du permis est-il domicilié dans la même commune ?
Koumoul_loge.loc[Koumoul_loge["COMMUNE_TERRAIN"]==Koumoul_loge["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
Koumoul_loge["DEM_MEME_COMMUNE"]=Koumoul_loge["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
Koumoul_loge["DEP_DEMANDEUR"]=Koumoul_loge["CP_DEMANDEUR"].str[:2]
Koumoul_loge["DEP_DEMANDEUR"]=Koumoul_loge["DEP_DEMANDEUR"].astype("str")
Koumoul_loge["DEP"]=Koumoul_loge["DEP"].astype("str")
Koumoul_loge["DEP"]=Koumoul_loge["DEP"].str[:2]
Koumoul_loge.loc[Koumoul_loge["DEP"]==Koumoul_loge["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
Koumoul_loge["DEM_MEME_DEP"]=Koumoul_loge["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne (à 5dep) ? 
dep_Bzh=("29","22","56","35","44")
Koumoul_loge.loc[Koumoul_loge["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
Koumoul_loge["DEM_BRETAGNE"]=Koumoul_loge["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
Koumoul_loge.loc[Koumoul_loge["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
Koumoul_loge["DEM_Ouest"]=Koumoul_loge["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
Koumoul_loge.loc[Koumoul_loge["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
Koumoul_loge["DEM_LOINTAINS"]=Koumoul_loge["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
Koumoul_loge["SIREN_EPCI"]=Koumoul_loge["SIREN_EPCI"].astype("str")
Koumoul_loge["EPCI_DEMANDEUR"]=Koumoul_loge["EPCI_DEMANDEUR"].astype("str")
Koumoul_loge.loc[Koumoul_loge["SIREN_EPCI"]==Koumoul_loge["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
Koumoul_loge["DEM_MEME_EPCI"]=Koumoul_loge["DEM_MEME_EPCI"].fillna("0")

Koumoul_loge.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
Koumoul_loge.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

Koumoul_loge["Type_AU"]="Logements"

Koumoul_loge=Koumoul_loge[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','NUM_PERMIS','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN','DESTINATION',
       'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW','NB_LOGEMENTS_NEW','TYPE_HABIT_PRINCIP_SECOND','RESID_SECOND',
       'CODE_ACTIV_DEMANDEUR','CAT_JURID_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3','latitude','longitude','parcel_confidence','parcelle']]

Koumoul_loge.info()


/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_9446/1413074225.py:1: DtypeWarning: Columns (22,30) have mixed types. Specify dtype option on import or set low_memory=False.
  Koumoul_loge=pd.read_csv("SITADEL/SITADEL_Koumoul_2014_2024/Koumoul_sitadel-logements.csv")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_9446/1413074225.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_9446/1413074225.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.s

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183206 entries, 0 to 183205
Data columns (total 46 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Type_AU                    183206 non-null  object        
 1   DEP                        183206 non-null  object        
 2   INSEE_COM                  183206 non-null  object        
 3   NOM_COMMUNE                183203 non-null  object        
 4   SIREN_EPCI                 183206 non-null  object        
 5   NOM_EPCI                   183170 non-null  object        
 6   NUM_PERMIS                 183206 non-null  object        
 7   DATE                       183206 non-null  object        
 8   ANNEE_MOIS_AUTORISATION    183206 non-null  datetime64[ns]
 9   PERMIS_ANNULE              183206 non-null  object        
 10  SUPERF_TERRAIN             183206 non-null  int64         
 11  DESTINATION                183206 non-null  int64   

## Locaux Sitadel

In [4]:
# Chargement de la liste des permis de construire de locaux :
Koumoul_Loc=pd.read_csv("SITADEL/SITADEL_Koumoul_2014_2024/Koumoul_sitadel-locaux.csv", low_memory=False)
Koumoul_Loc["DATE_REELLE_AUTORISATION"]=Koumoul_Loc["DATE_REELLE_AUTORISATION"].astype("str")

Koumoul_Loc=Koumoul_Loc.rename(columns={"COMM":"INSEE_COM","DEP_CODE":"DEP","DATE_REELLE_AUTORISATION":"DATE","NUM_DAU":"NUM_PERMIS","ETAT_DAU":"ETAT_AVANCEMENT",
    "CODPOST_DEM":"CP_DEMANDEUR","LOCALITE_DEM":"COMMUNE_DEMANDEUR","ADR_NUM_TER":"NUM_VOIE_TERRAIN",
    "ADR_TYPEVOIE_TER":"TYPE_VOIE-TERRAIN","ADR_LIBVOIE_TER":"NOM_VOIE_TERRAIN","ADR_CODPOST_TER":"CP_TERRAIN",
    "ADR_LOCALITE_TER":"COMMUNE_TERRAIN","SUPERFICIE_TERRAIN":"SUPERF_TERRAIN","NATURE_PROJET_DECLAREE":"NATURE_PROJET",
    "NATURE_PROJET_COMPLETEE":"NATURE_PROJET_DETAIL","DESTINATION_PRINCIPALE":"DESTINATION","SURF_HAB_CREEE":"SURF_HABIT_NEW",
    "SURF_LOC_CREEE":"SURF_LOCAUX_NEW", "SURF_HEB_CREEE":"SURF_HOTEL_NEW", "SURF_BUR_CREEE":"SURF_BUREAU_NEW",
    "SURF_COM_CREEE":"SURF_COMMERC_NEW","SURF_ART_CREEE":"SURF_ARTISAN_NEW","SURF_IND_CREEE":"SURF_INDUS_NEW",
    "SURF_AGR_CREEE":"SURF_AGRIC_NEW","SURF_ENT_CREEE":"SURF_ENTREPOT_NEW","SURF_PUB_CREEE":"SURF_SERV_PUB_NEW",
    "TYPE_SERVICE_PUBLIC":"TYPE_SERV_PUB","APE_DEM":"CODE_ACTIV_DEMANDEUR","CAT_DEM":"CAT_JURID_DEMANDEUR",
    "DENOM_DEM":"NOM_DEMANDEUR","SIREN_DEM":"SIREN_DEMANDEUR","SIRET_DEM":"SIRET_DEMANDEUR"})

Koumoul_Loc["NATURE_PROJET"]=Koumoul_Loc["NATURE_PROJET"].astype("str")
Koumoul_Loc=Koumoul_Loc[Koumoul_Loc["NATURE_PROJET"]=="1"]

#### Création d'une nouvelle variable permettant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
Koumoul_Loc.loc[Koumoul_Loc["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
Koumoul_Loc.loc[Koumoul_Loc["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
Koumoul_Loc.drop(columns="ETAT_AVANCEMENT", inplace=True)

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
Koumoul_Loc["DATE"]=Koumoul_Loc["DATE"].astype("str")
Koumoul_Loc["ANNEE_MOIS_AUTORISATION"]=(Koumoul_Loc["DATE"].str[:7])+"-01 01:00:00"
Koumoul_Loc["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(Koumoul_Loc["ANNEE_MOIS_AUTORISATION"])

# Modifier les codes insee des communes (qui ont pu changer depuis 2013 suite aux fusions de communes), 
# en utilisant la table de passage (https://www.insee.fr/fr/information/7671867):
Koumoul_Loc["INSEE_COM"]=Koumoul_Loc["INSEE_COM"].astype("str")
Koumoul_Loc.replace({"INSEE_COM": {"22154":"22093", "22173":"22093", "22038":"22206", "22100":"22219", "22078":"22264", "22247":"22264", "22253":"22264", "22123":"22050", "22007":"22055", "22167":"22107", "22290":"22107", "22298":"22158", "22192":"22209", "22357":"22209", "22066":"22046", "22102":"22046", "22191":"22046", "22292":"22046", "22297":"22046", "22303":"22046", "22051":"22084", "22151":"22093", "22058":"22183", "22080":"22203", "22367":"22251", "29219":"29199", "29129":"29227", "29203":"29021", "29149":"29076", "29052":"29003", "29127":"29266", "35112":"35062", "35113":"35004", "35303":"35004", "35341":"35004", "35100":"35163", "35301":"35184", "35053":"35220", "35269":"35282", "35293":"35282", "35348":"35282", "35011":"35292", "35147":"35308", "35344":"35308", "35209":"35069", "35254":"35069", "35048":"35168", "35083":"35191", "35323":"35191", "35267":"35257", "35158":"35060", "35129":"35176", "44160":"44003", "44017":"44180", "44093":"44180", "44191":"44180", "44219":"44180", "44040":"44005", "44059":"44021", "44008":"44029", "44181":"44087", "44004":"44163", "44011":"44213", "44034":"44213", "44147":"44213", "44060":"49160", "56049":"56213", "56059":"56102", "56138":"56165", "56016":"56173", "56183":"56033", "56038":"56061", "56064":"56061", "56142":"56144", "56192":"56144", "56037":"56197", "56187":"56197", "56150":"56251"}}, inplace=True)

# Ajout du statut et de l'Epci de rattachement des communes :
Koumoul_Loc["INSEE_COM"]=Koumoul_Loc["INSEE_COM"].astype("str")
Com_BZH=Com_BZH[["INSEE_COM","NOM_COMMUNE","SIREN_EPCI","NOM_EPCI"]]
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
Koumoul_Loc=Koumoul_Loc.merge(Com_BZH, how="left", on="INSEE_COM")
Koumoul_Loc.rename(columns={"DEP_x":"DEP"}, inplace=True)


#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)

# On extrait une liste propre des communes :
Detail_Communes=Koumoul_Loc[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok", "SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)

# Puis on fusionne avec la base Sitadel :
Koumoul_Loc=Koumoul_Loc.merge(Detail_Communes, how="left", on="COMMUNE_DEMANDEUR")
Koumoul_Loc.replace({"EPCI_DEMANDEUR": {"NR": "0"}}, inplace=True)
Koumoul_Loc["EPCI_DEMANDEUR"]=Koumoul_Loc["EPCI_DEMANDEUR"].fillna("0").astype("int64")

# Le demandeur du permis est-il domicilié dans la même commune ?
Koumoul_Loc.loc[Koumoul_Loc["COMMUNE_TERRAIN"]==Koumoul_Loc["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
Koumoul_Loc["DEM_MEME_COMMUNE"]=Koumoul_Loc["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
Koumoul_Loc["DEP_DEMANDEUR"]=Koumoul_Loc["CP_DEMANDEUR"].str[:2]
Koumoul_Loc["DEP_DEMANDEUR"]=Koumoul_Loc["DEP_DEMANDEUR"].astype("str")
Koumoul_Loc["DEP"]=Koumoul_Loc["DEP"].astype("str")
Koumoul_Loc["DEP"]=Koumoul_Loc["DEP"].str[:2]
Koumoul_Loc.loc[Koumoul_Loc["DEP"]==Koumoul_Loc["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
Koumoul_Loc["DEM_MEME_DEP"]=Koumoul_Loc["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne ? 
dep_Bzh=("29","22","56","35")
Koumoul_Loc.loc[Koumoul_Loc["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
Koumoul_Loc["DEM_BRETAGNE"]=Koumoul_Loc["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
Koumoul_Loc.loc[Koumoul_Loc["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
Koumoul_Loc["DEM_Ouest"]=Koumoul_Loc["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
Koumoul_Loc.loc[Koumoul_Loc["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
Koumoul_Loc["DEM_LOINTAINS"]=Koumoul_Loc["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
Koumoul_Loc["SIREN_EPCI"]=Koumoul_Loc["SIREN_EPCI"].astype("str")
Koumoul_Loc["EPCI_DEMANDEUR"]=Koumoul_Loc["EPCI_DEMANDEUR"].astype("str")
Koumoul_Loc.loc[Koumoul_Loc["SIREN_EPCI"]==Koumoul_Loc["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
Koumoul_Loc["DEM_MEME_EPCI"]=Koumoul_Loc["DEM_MEME_EPCI"].fillna("0")

Koumoul_Loc.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
Koumoul_Loc.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

Koumoul_Loc["Type_AU"]="Locaux"

Koumoul_Loc=Koumoul_Loc[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','NUM_PERMIS','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN',
       'NATURE_PROJET','NATURE_PROJET_DETAIL', 'DESTINATION',
       'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'SURF_HOTEL_NEW','SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW','SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 
       'CODE_ACTIV_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3','latitude','longitude','parcel_confidence','parcelle']]

Koumoul_Loc.info()


/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_9446/362191271.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_9446/362191271.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_9446/362191271.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35310 entries, 0 to 35309
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Type_AU                  35310 non-null  object        
 1   DEP                      35310 non-null  object        
 2   INSEE_COM                35310 non-null  object        
 3   NOM_COMMUNE              35309 non-null  object        
 4   SIREN_EPCI               35310 non-null  object        
 5   NOM_EPCI                 35284 non-null  object        
 6   NUM_PERMIS               35310 non-null  object        
 7   DATE                     35310 non-null  object        
 8   ANNEE_MOIS_AUTORISATION  35310 non-null  datetime64[ns]
 9   PERMIS_ANNULE            35310 non-null  object        
 10  SUPERF_TERRAIN           35310 non-null  int64         
 11  NATURE_PROJET            35310 non-null  object        
 12  NATURE_PROJET_DETAIL     35310 n

#### Croisement des permis avec la bande littorale, les zonages des PLU et la tâche artificialisée de 2014

In [24]:
# Chargement des polygones des bandes littorales :
BandeComLitt=gpd.read_file("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH_4326.geojson")
BandeComLitt=BandeComLitt[BandeComLitt["BANDE_LITT"]!="5_10km"]
BandeComLitt["ID_LITT"]=BandeComLitt["INSEE_COM"]+"_"+BandeComLitt["BANDE_LITT"]
BandeComLitt=BandeComLitt[['BANDE_LITT', 'ID_LITT','INSEE_COM','SURFACE_M2', 'geometry']]
SITADEL_Bzh=gpd.sjoin(SITADEL_Bzh, BandeComLitt, how="left")

SITADEL_Bzh=SITADEL_Bzh.merge(Com_BZH, on="INSEE_COM", how="left")
SITADEL_Bzh["DEP"]=SITADEL_Bzh["INSEE_COM"].str[:2]

SITADEL_Bzh["DATE"]=SITADEL_Bzh["DATE"].astype("str")
SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"]=SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"].astype("str")
SITADEL_Bzh["ANNEE"]=SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"].str[:4]


SITADEL_Bzh=SITADEL_Bzh[['Type_AU', 'DEP', 'INSEE_COM', 'NOM_COMMUNE',
       'SIREN_EPCI', 'NOM_EPCI', 'ID_LITT', 'BANDE_LITT','NUM_PERMIS', 'DATE',
       'ANNEE_MOIS_AUTORISATION', 'ANNEE', 'PERMIS_ANNULE', 'SUPERF_TERRAIN',
       'DESTINATION', 'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW',
       'TYPE_HABIT_PRINCIP_SECOND', 'RESID_SECOND', 'CODE_ACTIV_DEMANDEUR',
       'CAT_JURID_DEMANDEUR', 'NOM_DEMANDEUR', 'SIREN_DEMANDEUR',
       'SIRET_DEMANDEUR', 'DEP_DEMANDEUR', 'DEM_INSEE_COM',
       'NOM_COMMUNE_DEMANDEUR', 'EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP', 'DEM_BRETAGNE',
       'DEM_Ouest', 'DEM_LOINTAINS', 'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN',
       'NOM_VOIE_TERRAIN', 'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3','parcelle',
       'NATURE_PROJET', 'NATURE_PROJET_DETAIL', 'SURF_HOTEL_NEW',
       'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW',
       'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'geometry']]

######### filtrer les permis délivrés dans la bande des 5 km du littoral :
SITADEL_Bzh_Litt=SITADEL_Bzh[SITADEL_Bzh["BANDE_LITT"].notna()]

SITADEL_Bzh_Litt["Latitude"] = SITADEL_Bzh_Litt["geometry"].y
SITADEL_Bzh_Litt["Longitude"] = SITADEL_Bzh_Litt["geometry"].x


### Les permis de construire sont-ils attribués en dehors de la zone déjà urbanisée en 2014 ?

# Chargement des permis de construire géolocalisés (données SITADEL Koumoul 2014-2024)
SITADEL_Bzh_Litt=gpd.read_file("SITADEL/SITADEL_Bzh_Litt_2014-2024.geojson")
# Chargement des la tache artificialisée 2014 découpée par communes et bandes littorales :
TacheUrb2014=gpd.read_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_BandeLitt_DEF.geojson")

TacheUrb2014["geometry"]=TacheUrb2014["geometry"].to_crs("epsg:4326")
TacheUrb2014["TacheArtif2014"]="TacheUrb"
TacheUrb2014=TacheUrb2014[["geometry","TacheArtif2014"]]

# Jointure spatiale des deux fichiers et localisation des permis par rapport à la tâche urbaine de 2014 :
SITADEL_Bzh_New=gpd.sjoin(SITADEL_Bzh_Litt, TacheUrb2014, how="left")

SITADEL_Bzh_New.loc[SITADEL_Bzh_New["TacheArtif2014"].isna(), "TacheUrb2014"]="extérieur"
SITADEL_Bzh_New.loc[SITADEL_Bzh_New["TacheArtif2014"]=="TacheUrb", "TacheUrb2014"]="intérieur"

SITADEL_Bzh_New=SITADEL_Bzh_New[['Type_AU', 'DEP', 'INSEE_COM', 'NOM_COMMUNE',
       'SIREN_EPCI', 'NOM_EPCI', 'ID_LITT', 'BANDE_LITT', 'TacheUrb2014', 'NUM_PERMIS', 'DATE',
       'ANNEE_MOIS_AUTORISATION', 'ANNEE', 'PERMIS_ANNULE', 'SUPERF_TERRAIN',
       'DESTINATION', 'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW',
       'TYPE_HABIT_PRINCIP_SECOND', 'RESID_SECOND', 'CODE_ACTIV_DEMANDEUR',
       'CAT_JURID_DEMANDEUR', 'NOM_DEMANDEUR', 'SIREN_DEMANDEUR',
       'SIRET_DEMANDEUR', 'DEP_DEMANDEUR', 'DEM_INSEE_COM',
       'NOM_COMMUNE_DEMANDEUR', 'EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP', 'DEM_BRETAGNE',
       'DEM_Ouest', 'DEM_LOINTAINS','parcelle', 'NATURE_PROJET', 'NATURE_PROJET_DETAIL', 'SURF_HOTEL_NEW',
       'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW',
       'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'Latitude', 'Longitude',
       'geometry']]

### Dans quel secteur des Plans locaux d'urbanisme (actuellement en vigueur) chaque permis de construire est-il situé ?

# Chargement des secteurs de documents d'urbanisme :
ZonesUrba=gpd.read_file("Zones_Urba_ComLitt_Bzh_new.shp")[["ZONE","libelle","libelong","SOURCE","geometry"]]
ZonesUrba.rename(columns={"ZONE":"ZoneUrba","libelle":"CodeZoneUrba","libelong":"NomZoneUrba", "SOURCE":"SourceZoneUrba"}, inplace=True)

# Jointure spatiale des deux fichiers et sélection des permis localisés en dehors de la tâche urbaine de 2014 :
SITADEL_Bzh_New=gpd.sjoin(SITADEL_Bzh_New, ZonesUrba, how="left")

SITADEL_Bzh_New=SITADEL_Bzh_New[[
    'Type_AU', 'DEP', 'INSEE_COM','NOM_COMMUNE','SIREN_EPCI', 'NOM_EPCI', 'ID_LITT','BANDE_LITT', 'TacheUrb2014','ZoneUrba', 
    'CodeZoneUrba','NomZoneUrba','SourceZoneUrba','NUM_PERMIS', 'DATE', 'ANNEE_MOIS_AUTORISATION', 'ANNEE',
    'PERMIS_ANNULE', 'SUPERF_TERRAIN', 'DESTINATION', 'SURF_HABIT_NEW','SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW', 
    'TYPE_HABIT_PRINCIP_SECOND','RESID_SECOND', 'CODE_ACTIV_DEMANDEUR', 'CAT_JURID_DEMANDEUR','NOM_DEMANDEUR', 
    'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR', 'DEP_DEMANDEUR','DEM_INSEE_COM', 'NOM_COMMUNE_DEMANDEUR',
    'EPCI_DEMANDEUR', 'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP','DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
    'parcelle', 'NATURE_PROJET', 'NATURE_PROJET_DETAIL','SURF_HOTEL_NEW', 'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW','SURF_ARTISAN_NEW', 
    'SURF_INDUS_NEW', 'SURF_AGRIC_NEW','SURF_ENTREPOT_NEW', 'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'Latitude','Longitude', 'geometry'
    ]]

SITADEL_Bzh_New.to_file("SITADEL/SITADEL_Bzh_Litt_2014-2024.geojson")

SITADEL_Bzh_New_Plat=SITADEL_Bzh_New[[
    'Type_AU', 'DEP', 'INSEE_COM','NOM_COMMUNE','SIREN_EPCI', 'NOM_EPCI', 'ID_LITT','BANDE_LITT', 'TacheUrb2014','ZoneUrba', 
    'CodeZoneUrba','NomZoneUrba','SourceZoneUrba','NUM_PERMIS', 'DATE', 'ANNEE_MOIS_AUTORISATION', 'ANNEE',
    'PERMIS_ANNULE', 'SUPERF_TERRAIN', 'DESTINATION', 'SURF_HABIT_NEW','SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW', 
    'TYPE_HABIT_PRINCIP_SECOND','RESID_SECOND', 'CODE_ACTIV_DEMANDEUR', 'CAT_JURID_DEMANDEUR','NOM_DEMANDEUR', 
    'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR', 'DEP_DEMANDEUR','DEM_INSEE_COM', 'NOM_COMMUNE_DEMANDEUR',
    'EPCI_DEMANDEUR', 'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP','DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
    'parcelle', 'NATURE_PROJET', 'NATURE_PROJET_DETAIL','SURF_HOTEL_NEW', 'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW','SURF_ARTISAN_NEW', 
    'SURF_INDUS_NEW', 'SURF_AGRIC_NEW','SURF_ENTREPOT_NEW', 'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'Latitude','Longitude'
    ]]

SITADEL_Bzh_New.info()

/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/pyogrio/raw.py:196: RuntimeWarning: Zones_Urba_ComLitt_Bzh_new.s

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 76776 entries, 0 to 76771
Data columns (total 55 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Type_AU                    76776 non-null  object        
 1   DEP                        76776 non-null  object        
 2   INSEE_COM                  76776 non-null  object        
 3   NOM_COMMUNE                76776 non-null  object        
 4   SIREN_EPCI                 76776 non-null  object        
 5   NOM_EPCI                   76724 non-null  object        
 6   ID_LITT                    76776 non-null  object        
 7   BANDE_LITT                 76776 non-null  object        
 8   TacheUrb2014               76776 non-null  object        
 9   ZoneUrba                   58167 non-null  object        
 10  CodeZoneUrba               58167 non-null  object        
 11  NomZoneUrba                58167 non-null  object        
 12  S

#### Synthèse par communes

In [26]:
SITADEL_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["INSEE_COM","NOM_COMMUNE","SIREN_EPCI"], as_index=False).agg(
   Nb_Permis=("INSEE_COM","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum")
   )

SITADEL_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Litt_Bilan["Permis_annules"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan=SITADEL_Litt_Bilan[['INSEE_COM','NOM_COMMUNE','SIREN_EPCI', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Litt_Bilan.to_csv("SITADEL/SITADEL_Complet_Bzh_Bilan_COM.csv")
SITADEL_Litt_Bilan.sample(5)

,INSEE_COM,NOM_COMMUNE,SIREN_EPCI,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
14,22093,Lamballe-Armor,200069391,217,4.000000000,30.000000000,52.000000000,82.000000000,86.000000000,89.000000000,11.000000000,209,3,1350198.000000000,22341.000000000,15441.000000000,30,103,2001,0,60,12907,121,219
111,29039,Concarneau,242900769,1008,4.000000000,48.000000000,62.000000000,86.000000000,92.000000000,92.000000000,8.000000000,2117,88,2713939.000000000,192663.000000000,75306.000000000,2407,11532,11267,4470,19313,1106,7105,16625
209,29220,Pont-l'Abbé,242900702,410,3.000000000,31.000000000,68.000000000,87.000000000,91.000000000,92.000000000,8.000000000,714,26,1070762.000000000,76708.000000000,31287.000000000,0,4497,5007,2320,1773,3339,2288,10413
306,44131,Pornic,200067346,1594,2.000000000,12.000000000,49.000000000,82.000000000,77.000000000,89.000000000,11.000000000,2844,136,3447174.000000000,277022.000000000,121966.000000000,18660,15485,35041,7438,3515,21923,6567,8620
54,22224,Ploulec'h,200065928,63,5.000000000,30.000000000,83.000000000,92.000000000,97.000000000,97.000000000,3.000000000,64,0,137760.000000000,6591.000000000,8175.000000000,0,49,1935,0,9,2700,399,2990


#### Comparaison du volume de permis délivrés à l'intérieur et en dehors de la zone déjà artificialisé en 2014, selon la bande littorale

In [27]:
SITADEL_Bzh_Litt=gpd.read_file("SITADEL/SITADEL_Bzh_Litt_2014-2024.geojson")

SITADEL_Bzh_Litt.replace({"PERMIS_ANNULE": {"non": "0", "oui": "1"}}, inplace=True)
SITADEL_Bzh_Litt["PERMIS_ANNULE"]=SITADEL_Bzh_Litt["PERMIS_ANNULE"].astype("int64")

SITADEL_Bzh_Litt.replace({"RESID_SECOND": {"0.0": "0", "":"0", "1.0": "1"}}, inplace=True)
SITADEL_Bzh_Litt["RESID_SECOND"]=SITADEL_Bzh_Litt["RESID_SECOND"].fillna("0").astype("int64")

SITADEL_Bzh_Litt["SUPERF_TERRAIN"]=SITADEL_Bzh_Litt["SUPERF_TERRAIN"].astype("float64")
SITADEL_Bzh_Litt["SURF_HABIT_NEW"]=SITADEL_Bzh_Litt["SURF_HABIT_NEW"].astype("float64")
SITADEL_Bzh_Litt["SURF_LOCAUX_NEW"]=SITADEL_Bzh_Litt["SURF_LOCAUX_NEW"].astype("float64")
SITADEL_Bzh_Litt["SURF_HOTEL_NEW"]=SITADEL_Bzh_Litt["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_BUREAU_NEW"]=SITADEL_Bzh_Litt["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_COMMERC_NEW"]=SITADEL_Bzh_Litt["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_ARTISAN_NEW"]=SITADEL_Bzh_Litt["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_INDUS_NEW"]=SITADEL_Bzh_Litt["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_AGRIC_NEW"]=SITADEL_Bzh_Litt["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_Litt["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_Litt["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_COMMUNE"]=SITADEL_Bzh_Litt["DEM_MEME_COMMUNE"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_EPCI"]=SITADEL_Bzh_Litt["DEM_MEME_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_DEP"]=SITADEL_Bzh_Litt["DEM_MEME_DEP"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_BRETAGNE"]=SITADEL_Bzh_Litt["DEM_BRETAGNE"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_Ouest"]=SITADEL_Bzh_Litt["DEM_Ouest"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_LOINTAINS"]=SITADEL_Bzh_Litt["DEM_LOINTAINS"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_Litt["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["TacheUrb2014","BANDE_LITT"], as_index=False).agg(
   Nb_Permis=("Type_AU","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['TacheUrb2014','BANDE_LITT', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_Bzh_Bilan_Artif_NEW-OLD_2013_2024.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)

,TacheUrb2014,BANDE_LITT,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
3,extérieur,200_500m,3148,5.000000000,25.000000000,46.000000000,74.000000000,82.000000000,86.000000000,14.000000000,4521,367,12827503.000000000,498020.000000000,161880.000000000,21094,19013,16882,6020,13173,40781,8227,17633
2,extérieur,1_5km,20444,4.000000000,28.000000000,55.000000000,82.000000000,87.000000000,91.000000000,9.000000000,26045,975,103097624.000000000,2607127.000000000,3007204.000000000,41770,260576,213842,244275,307801,1344802,342420,215112
8,intérieur,200_500m,8995,6.000000000,25.000000000,43.000000000,69.000000000,78.000000000,82.000000000,18.000000000,18514,1484,17276955.000000000,1922582.000000000,350839.000000000,7233,51065,57014,17309,32048,48187,20605,70368
6,intérieur,100m,1599,5.000000000,28.000000000,45.000000000,71.000000000,76.000000000,82.000000000,18.000000000,3634,294,5186485.000000000,410869.000000000,184141.000000000,36676,19073,21634,3657,15701,8114,8282,33470
7,intérieur,1_5km,21070,5.000000000,33.000000000,57.000000000,80.000000000,84.000000000,87.000000000,13.000000000,40979,1004,68632602.000000000,3833752.000000000,1979035.000000000,48241,288258,335097,161542,111856,366699,269279,301732


---------------------------------

## L'artificialisation à venir
(Permis délivrés depuis janvier 2020 en dehors des zones déjà artificialisées en 2024)


In [59]:
# Filtrage et export des permis délivrés depuis le 1er janvier 2020 (et donc potentiellement valides jusqu'en janvier 2025), 
# en dehors du secteurs déjà artificialisé en 2024 :
SITADEL_Bzh_Litt=gpd.read_file("SITADEL/SITADEL_Bzh_Litt_2014-2024.geojson")

SITADEL_Bzh_New2024=SITADEL_Bzh_Litt[SITADEL_Bzh_Litt["ANNEE_MOIS_AUTORISATION"]>="2020-01-01 01:00:00"]
SITADEL_Bzh_New2024=SITADEL_Bzh_New2024[SITADEL_Bzh_New2024["TacheUrb2014"]=="extérieur"]

# Chargement de la tache artificialisée 2024 découpée par communes et bandes littorales :
TacheUrb2024=gpd.read_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_2024_BandeLitt_DEF.geojson")
TacheUrb2024["geometry"]=TacheUrb2024["geometry"].to_crs("epsg:4326")
TacheUrb2024["TacheArtif2024"]="TacheUrb"
TacheUrb2024=TacheUrb2024[["geometry","TacheArtif2024"]]

# Jointure spatiale des deux fichiers et sélection des permis localisés en dehors de la tâche urbaine de 2024 :
SITADEL_Bzh_New2024=gpd.sjoin(SITADEL_Bzh_New2024, TacheUrb2024, how="left")
SITADEL_Bzh_New2024.loc[SITADEL_Bzh_New2024["TacheArtif2024"].isna(), "TacheUrb2024"]="extérieur"


# Préparer les variables pour les synthèses :
SITADEL_Bzh_New2024.replace({"PERMIS_ANNULE": {"non": "0", "oui": "1"}}, inplace=True)
SITADEL_Bzh_New2024["PERMIS_ANNULE"]=SITADEL_Bzh_New2024["PERMIS_ANNULE"].astype("int64")
SITADEL_Bzh_New2024.replace({"RESID_SECOND": {"0.0": "0", "":"0", "1.0": "1"}}, inplace=True)
SITADEL_Bzh_New2024["RESID_SECOND"]=SITADEL_Bzh_New2024["RESID_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SUPERF_TERRAIN"]=SITADEL_Bzh_New2024["SUPERF_TERRAIN"].astype("float64")
SITADEL_Bzh_New2024["SURF_HABIT_NEW"]=SITADEL_Bzh_New2024["SURF_HABIT_NEW"].astype("float64")
SITADEL_Bzh_New2024["SURF_LOCAUX_NEW"]=SITADEL_Bzh_New2024["SURF_LOCAUX_NEW"].astype("float64")
SITADEL_Bzh_New2024["SURF_HOTEL_NEW"]=SITADEL_Bzh_New2024["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_BUREAU_NEW"]=SITADEL_Bzh_New2024["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_COMMERC_NEW"]=SITADEL_Bzh_New2024["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_ARTISAN_NEW"]=SITADEL_Bzh_New2024["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_INDUS_NEW"]=SITADEL_Bzh_New2024["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_AGRIC_NEW"]=SITADEL_Bzh_New2024["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_New2024["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_New2024["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_MEME_COMMUNE"]=SITADEL_Bzh_New2024["DEM_MEME_COMMUNE"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_MEME_EPCI"]=SITADEL_Bzh_New2024["DEM_MEME_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_MEME_DEP"]=SITADEL_Bzh_New2024["DEM_MEME_DEP"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_BRETAGNE"]=SITADEL_Bzh_New2024["DEM_BRETAGNE"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_Ouest"]=SITADEL_Bzh_New2024["DEM_Ouest"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_LOINTAINS"]=SITADEL_Bzh_New2024["DEM_LOINTAINS"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_New2024["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")



SITADEL_Bzh_New2024=SITADEL_Bzh_New2024[SITADEL_Bzh_New2024["TacheUrb2024"]=="extérieur"]
SITADEL_Bzh_New2024=SITADEL_Bzh_New2024[SITADEL_Bzh_New2024["PERMIS_ANNULE"]==0]

SITADEL_Bzh_New2024=SITADEL_Bzh_New2024.reset_index()
SITADEL_Bzh_New2024.rename(columns={"index":"ID_PERMIS"}, inplace=True)



#### Synthèses

In [61]:
######### SYNTHESE DES PERMIS PAR BANDE LITTORALE et par commune ##########

SITADEL_Bzh_New2024_Bilan=SITADEL_Bzh_New2024.groupby(["INSEE_COM","NOM_COMMUNE","BANDE_LITT"], as_index=False).agg(Nb_Permis=("ID_LITT","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_New2024_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_New2024_Bilan["Permis_annules"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_New2024_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Bzh_New2024_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_New2024_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_New2024_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_New2024_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_New2024_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_New2024_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_New2024_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_New2024_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_New2024_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_New2024_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_New2024_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_New2024_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_New2024_Bilan=SITADEL_Bzh_New2024_Bilan[['INSEE_COM','NOM_COMMUNE', 'BANDE_LITT', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_New2024_Bilan.to_csv("SITADEL/SITADEL_Bzh_Bilan2024_COM_BandeLitt.csv")
SITADEL_Bzh_New2024_Bilan.sample(5)

,INSEE_COM,NOM_COMMUNE,BANDE_LITT,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
37,22162,Paimpol,200_500m,2,0.000000000,0.000000000,0.000000000,100.000000000,100.000000000,100.000000000,0.000000000,1,0,7456.000000000,148.000000000,3326.000000000,0,0,0,0,0,0,0,3326
597,56034,Carnac,1_5km,7,0.000000000,57.000000000,71.000000000,100.000000000,100.000000000,100.000000000,0.000000000,22,0,145916.000000000,1668.000000000,3211.000000000,0,90,2353,0,0,768,0,0
725,56240,Sarzeau,1_5km,35,0.000000000,46.000000000,69.000000000,74.000000000,91.000000000,91.000000000,9.000000000,26,4,527779.000000000,2962.000000000,10346.000000000,0,550,2208,561,636,3074,890,2411
144,22379,Trévou-Tréguignec,100m,1,0.000000000,0.000000000,0.000000000,100.000000000,100.000000000,100.000000000,0.000000000,1,0,397.000000000,87.000000000,0.000000000,0,0,0,0,0,0,0,0
223,29072,Guilvinec,200_500m,2,0.000000000,0.000000000,0.000000000,0.000000000,50.000000000,50.000000000,50.000000000,1,0,3396.000000000,118.000000000,1.000000000,0,0,0,0,0,0,0,1
